In [ ]:



!pip install transformers accelerate peft datasets torch bitsandbytes pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/glaive-python-code-qa-dataset")

print("Path to dataset files:", path)

Dataset URL: https://www.kaggle.com/datasets/thedevastator/glaive-python-code-qa-dataset
License(s): CC0-1.0
 98% 58.0M/59.3M [00:03<00:00, 23.5MB/s]
100% 59.3M/59.3M [00:03<00:00, 16.5MB/s]


In [ ]:
def load_csv_to_dataset(csv_path="/content/train.csv"):

    df = pd.read_csv(csv_path)

    # Ensure 'text' column exists, else combine relevant columns
    if "text" not in df.columns:
        df["text"] = df.apply(lambda row: " ".join(str(row[col]) for col in df.columns), axis=1)

    # Convert to Hugging Face dataset format
    dataset = Dataset.from_pandas(df[["text"]])
    return dataset


In [ ]:
def tokenize_dataset(dataset):
    tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset, tokenizer


In [ ]:
def fine_tune_model(tokenized_dataset, tokenizer):
    model = AutoModelForCausalLM.from_pretrained(
        "deepseek-ai/deepseek-r1-7b",
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # Apply LoRA for efficient fine-tuning
    lora_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM")
    model = get_peft_model(model, lora_config)

    training_args = TrainingArguments(
        output_dir="./deepseek_college",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        evaluation_strategy="steps",
        save_strategy="steps",
        save_steps=1000,
        logging_dir="./logs",
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        fp16=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset
    )

    trainer.train()

    # Save fine-tuned model
    model.save_pretrained("./deepseek_college_tuned")
    tokenizer.save_pretrained("./deepseek_college_tuned")
    return model


In [ ]:
def generate_response(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
print("AI:", generate_response(model, tokenizer, "Tell me about my semester subjects."))


NameError: name 'model' is not defined

In [ ]:
# Load CSV data
dataset = load_csv_to_dataset()

# Tokenize data
tokenized_dataset, tokenizer = tokenize_dataset(dataset)

# Fine-tune the model
model = fine_tune_model(tokenized_dataset, tokenizer)

# Test AI
response = generate_response(model, tokenizer, "What is the hostel curfew time?")
print("AI:", response)


ParserError: Error tokenizing data. C error: EOF inside string starting at row 9425